# Práctica clasificación Smartports


### Antonio Serrano Rodriguez, Guillermo García Engelmo

## Transformaciones de imágenes

In [1]:
from dataset import DatasetReader
import torch
import torchvision.transforms as transforms

# Estadísticas de ImageNet para normalizar
imagenet_mean = (0.485, 0.456, 0.406)
imagenet_std  = (0.229, 0.224, 0.225)

# Transformaciones “base” (redimensionar y normalizar)  
regnettransforms = transforms.Compose([
    transforms.Resize((224, 224)),                   
    transforms.ToTensor(),                           
    transforms.Normalize(imagenet_mean, imagenet_std) 
])

# Transformaciones para data augmentation 
aug_transforms = transforms.Compose([
    # Varias escalas y recortes aleatorios según proporciones reales del dataset 
    transforms.RandomResizedCrop(
        224,
        scale=(0.7, 1.0),    # barcos muy lejos (70%) o casi completos (100%)
        ratio=(0.8, 1.2)     # relación de aspecto ligeramente variable
    ),

    # Ángulos y perspectiva 
    transforms.RandomRotation(15),                   
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),          

    # Iluminación y color (amaneceres, atardeceres, sombras…) 
    transforms.ColorJitter(
        brightness=0.5,
        contrast=0.5,
        saturation=0.3,
        hue=0.1
    ),

    # Simular desenfoque típico de las cámaras
    transforms.RandomApply(
        [transforms.GaussianBlur(kernel_size=3)],
        p=0.3
    ),

    # Aplastamos al tamaño de entrada y normalizamos como ImageNet 
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std),

    # Oclusiones parciales para robustez ante muelles, niebla, etc. 
    transforms.RandomErasing(
        p=0.2,
        scale=(0.02, 0.15),
        ratio=(0.3, 3.3)
    )
])


## Preparación de datos

- **Datasets**  
  - `ShipsDs` / `ShipsDs_aug`: imágenes de barcos con transforms base y augmentadas.  
  - `DockDs` / `DockDs_aug`: lo mismo para barcos atracados.

- **Split**  
  - Test: 20 %, Train: 80 %.  
  - Semilla fija (`torch.manual_seed(42)`) para reproducibilidad.

- **Particiones**  
  - `random_split` en train/test para ambos datasets (con y sin augment).

- **DataLoaders**  
  - Batch size = 64.  
  - Shuffle en train, no shuffle en test.  
  - Cargadores por separado para versiones augmentadas y no augmentadas.  


In [2]:
# Crear datasets 
ShipsDs = DatasetReader("materiales/ship.csv", "materiales/images", transform= regnettransforms)
ShipsDs_aug = DatasetReader("materiales/ship.csv", "materiales/images", transform= aug_transforms)

DockDs = DatasetReader("materiales/docked.csv", "materiales/images", transform= regnettransforms)
DockDs_aug = DatasetReader("materiales/docked.csv", "materiales/images", transform= aug_transforms)

TEST_RATIO = 0.2
BATCH_SIZE = 64

num_train = int((1.0 - TEST_RATIO) * len(ShipsDs))
num_test = len(ShipsDs) - num_train

dc_num_train = int((1.0 - TEST_RATIO) * len(DockDs))
dc_num_test = len(DockDs) - dc_num_train

# Partitions
torch.manual_seed(42)

train_set, test_set = torch.utils.data.random_split(ShipsDs, [num_train, num_test])
train_set_aug, _ = torch.utils.data.random_split(ShipsDs_aug, [num_train, num_test])

# For docked dataset
dc_train_set, dc_test_set = torch.utils.data.random_split(DockDs, [dc_num_train, dc_num_test])
dc_train_set_aug, _ = torch.utils.data.random_split(DockDs_aug, [dc_num_train, dc_num_test])

# Loaders for Ships
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                          shuffle=False)

train_loader_aug = torch.utils.data.DataLoader(train_set_aug, batch_size=BATCH_SIZE,
                                            shuffle=True)

# Loaders for docks
dc_train_loader = torch.utils.data.DataLoader(dc_train_set, batch_size=BATCH_SIZE,
                                            shuffle=True)
dc_test_loader = torch.utils.data.DataLoader(dc_test_set, batch_size=BATCH_SIZE,
                                          shuffle=False)

dc_train_loader_aug = torch.utils.data.DataLoader(dc_train_set_aug, batch_size=BATCH_SIZE,
                                            shuffle=True)


## Definición de modelos

- **Función `make_regnet(pretrained)`**  
  - Carga `regnet_y_400mf` con o sin pesos (`pretrained=True/False`).  
  - Reemplaza la capa final con un clasificador de 3 capas:  →256 →128 →2.

- **Variantes por tarea**  
  - **Ship / No Ship**:  
    - `ship_scratch_noaug`, `ship_scratch_aug` (sin pesos).  
    - `ship_pretrained_noaug`, `ship_pretrained_aug` (con pesos).  
  - **Docked / Undocked**:  
    - `dock_scratch_noaug`, `dock_scratch_aug` (sin pesos).  
    - `dock_pretrained_noaug`, `dock_pretrained_aug` (con pesos).

- **Agrupación**  
  - Listas `ship_models` y `dock_models` para iterar fácilmente.  


In [3]:
from torchvision import models
import torch.nn as nn

# Función para crear y cambiar el final de una RegNet
def make_regnet(pretrained: bool):
    # pretrained=False  no carga pesos, True: carga los pesos
    model = models.regnet_y_400mf(pretrained=pretrained)
    in_feats = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(in_feats, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 2)      # salida binaria
    )
    return model

# Creamos nuestros 4 “tipos” de modelo para cada tarea
# Scratch: sin pesos
# Pretrained: con pesos
# aug: con data augmentation
# noaug: sin data augmentation

# Ship / No‑Ship
ship_scratch_noaug    = make_regnet(pretrained=False)
ship_scratch_aug      = make_regnet(pretrained=False)
ship_pretrained_noaug = make_regnet(pretrained=True)
ship_pretrained_aug   = make_regnet(pretrained=True)

# Docked / Undocked
dock_scratch_noaug    = make_regnet(pretrained=False)
dock_scratch_aug      = make_regnet(pretrained=False)
dock_pretrained_noaug = make_regnet(pretrained=True)
dock_pretrained_aug   = make_regnet(pretrained=True)

# Agrupamos en listas para iterar fácilmente
ship_models = [
    ship_scratch_noaug,
    ship_scratch_aug,
    ship_pretrained_noaug,
    ship_pretrained_aug
]
dock_models = [
    dock_scratch_noaug,
    dock_scratch_aug,
    dock_pretrained_noaug,
    dock_pretrained_aug
]




c:\Users\guill\Desktop\universidad\tercero\VCA\practicas\vca\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\guill\Desktop\universidad\tercero\VCA\practicas\vca\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
c:\Users\guill\Desktop\universidad\tercero\VCA\practicas\vca\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RegNet_Y_400MF_Weights.IMAGENET1K_V1`. You can also use `weights=RegNet_Y_400MF_Weights.DEFAULT` to get the most 

## Bucle de entrenamiento

In [4]:
from torch import optim
import torch.nn as nn
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train(model, train_loader, num_epochs, model_name="model", lr=1e-4):
    """
    Entrena durante num_epochs y guarda el modelo al final.
    """
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss, running_acc = 0.0, 0.0

        for samples, targets in train_loader:
            samples, targets = samples.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(samples)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_acc  += (outputs.argmax(1) == targets).float().mean().item()

        train_loss = running_loss / len(train_loader)
        train_acc  = running_acc  / len(train_loader)
        print(f"[Train] Epoch {epoch}/{num_epochs} — Loss: {train_loss:.4f}, Acc: {train_acc*100:.2f}%")

    # Al terminar todas las épocas, guardamos el modelo
    save_path = f"{model_name}.pth"
    torch.save(model.state_dict(), save_path)
    print(f"Modelo final guardado en {save_path}")
    return model


In [5]:
def test_model(model, test_loader, model_name="model"):
    #  Cargamos el modelo
    checkpoint = torch.load(f"{model_name}.pth", map_location=device)
    model.load_state_dict(checkpoint)

    # Evaluamos
    model = model.to(device)
    model.eval()
    criterion = nn.CrossEntropyLoss()

    total_correct = 0
    total_samples = 0
    total_loss    = 0.0

    with torch.no_grad():
        for samples, targets in test_loader:
            samples, targets = samples.to(device), targets.to(device)
            outputs = model(samples)
            total_loss    += criterion(outputs, targets).item()
            preds          = outputs.argmax(dim=1)
            total_correct += (preds == targets).sum().item()
            total_samples += targets.size(0)

    avg_loss = total_loss / len(test_loader)
    avg_acc  = total_correct / total_samples

    print(f"[Test] Loss: {avg_loss:.4f}, Acc: {avg_acc*100:.2f}%")


In [6]:
# Parámetros de los entrenamientos
NUM_EPOCHS = 10
LR         = 1e-4

## Modelos sobre detección de barcos o su ausencia

In [7]:
print("\n Modelo sin pesos y sin data augmentation")
train(ship_scratch_noaug, train_loader,      NUM_EPOCHS, model_name="ship_scratch_noaug", lr=LR)
test_model   (ship_scratch_noaug, test_loader,       model_name="ship_scratch_noaug")


 Modelo sin pesos y sin data augmentation
[Train] Epoch 1/10 — Loss: 0.6703, Acc: 61.02%
[Train] Epoch 2/10 — Loss: 0.6395, Acc: 61.78%
[Train] Epoch 3/10 — Loss: 0.6256, Acc: 61.40%
[Train] Epoch 4/10 — Loss: 0.6022, Acc: 63.54%
[Train] Epoch 5/10 — Loss: 0.5653, Acc: 66.27%
[Train] Epoch 6/10 — Loss: 0.5250, Acc: 71.16%
[Train] Epoch 7/10 — Loss: 0.4643, Acc: 81.29%
[Train] Epoch 8/10 — Loss: 0.3919, Acc: 88.29%
[Train] Epoch 9/10 — Loss: 0.3309, Acc: 90.85%
[Train] Epoch 10/10 — Loss: 0.2501, Acc: 93.18%
Modelo final guardado en ship_scratch_noaug.pth
[Test] Loss: 0.9741, Acc: 32.20%


In [8]:
print("\n Modelo sin pesos y con data augmentation\n")
train(ship_scratch_aug,   train_loader_aug,  NUM_EPOCHS, model_name="ship_scratch_aug",   lr=LR)
test_model   (ship_scratch_aug,    test_loader,       model_name="ship_scratch_aug")


 Modelo sin pesos y con data augmentation

[Train] Epoch 1/10 — Loss: 0.6599, Acc: 63.54%
[Train] Epoch 2/10 — Loss: 0.6607, Acc: 63.54%
[Train] Epoch 3/10 — Loss: 0.6509, Acc: 63.93%
[Train] Epoch 4/10 — Loss: 0.6546, Acc: 63.16%
[Train] Epoch 5/10 — Loss: 0.6587, Acc: 63.74%
[Train] Epoch 6/10 — Loss: 0.6545, Acc: 63.94%
[Train] Epoch 7/10 — Loss: 0.6523, Acc: 64.12%
[Train] Epoch 8/10 — Loss: 0.6601, Acc: 62.97%
[Train] Epoch 9/10 — Loss: 0.6464, Acc: 63.74%
[Train] Epoch 10/10 — Loss: 0.6545, Acc: 62.97%
Modelo final guardado en ship_scratch_aug.pth
[Test] Loss: 0.6320, Acc: 67.80%


In [9]:
print("\n Modelo con pesos y sin data augmentation\n")
train(ship_pretrained_noaug, train_loader,   NUM_EPOCHS, model_name="ship_pretrained_noaug", lr=LR)
test_model   (ship_pretrained_noaug, test_loader,    model_name="ship_pretrained_noaug")


 Modelo con pesos y sin data augmentation

[Train] Epoch 1/10 — Loss: 0.6551, Acc: 74.42%
[Train] Epoch 2/10 — Loss: 0.5421, Acc: 97.66%
[Train] Epoch 3/10 — Loss: 0.4493, Acc: 97.67%
[Train] Epoch 4/10 — Loss: 0.3468, Acc: 98.83%
[Train] Epoch 5/10 — Loss: 0.2584, Acc: 99.22%
[Train] Epoch 6/10 — Loss: 0.1785, Acc: 100.00%
[Train] Epoch 7/10 — Loss: 0.1096, Acc: 100.00%
[Train] Epoch 8/10 — Loss: 0.0633, Acc: 100.00%
[Train] Epoch 9/10 — Loss: 0.0404, Acc: 100.00%
[Train] Epoch 10/10 — Loss: 0.0230, Acc: 100.00%
Modelo final guardado en ship_pretrained_noaug.pth
[Test] Loss: 0.1580, Acc: 94.92%


In [10]:
print("\n Modelo con pesos y con data augmentation\n")
train(ship_pretrained_aug,   train_loader_aug, NUM_EPOCHS, model_name="ship_pretrained_aug",   lr=LR)
test_model   (ship_pretrained_aug,    test_loader,      model_name="ship_pretrained_aug")


 Modelo con pesos y con data augmentation

[Train] Epoch 1/10 — Loss: 0.6556, Acc: 63.74%
[Train] Epoch 2/10 — Loss: 0.5835, Acc: 63.55%
[Train] Epoch 3/10 — Loss: 0.5403, Acc: 63.57%
[Train] Epoch 4/10 — Loss: 0.4568, Acc: 73.85%
[Train] Epoch 5/10 — Loss: 0.3798, Acc: 88.09%
[Train] Epoch 6/10 — Loss: 0.3179, Acc: 93.37%
[Train] Epoch 7/10 — Loss: 0.2591, Acc: 96.29%
[Train] Epoch 8/10 — Loss: 0.2023, Acc: 96.49%
[Train] Epoch 9/10 — Loss: 0.1541, Acc: 96.68%
[Train] Epoch 10/10 — Loss: 0.1293, Acc: 97.08%
Modelo final guardado en ship_pretrained_aug.pth
[Test] Loss: 0.1180, Acc: 96.61%


## Modelos sobre detección de barcos atracados o navegando

In [11]:
print("\n Modelo sin pesos y sin data augmentation\n")
train(dock_scratch_noaug,    dc_train_loader,     NUM_EPOCHS, model_name="dock_scratch_noaug",    lr=LR)
test_model   (dock_scratch_noaug,    dc_test_loader,      model_name="dock_scratch_noaug")


 Modelo sin pesos y sin data augmentation

[Train] Epoch 1/10 — Loss: 0.6948, Acc: 56.33%
[Train] Epoch 2/10 — Loss: 0.6794, Acc: 57.98%
[Train] Epoch 3/10 — Loss: 0.6653, Acc: 63.29%
[Train] Epoch 4/10 — Loss: 0.6722, Acc: 60.77%
[Train] Epoch 5/10 — Loss: 0.6402, Acc: 65.19%
[Train] Epoch 6/10 — Loss: 0.6222, Acc: 67.79%
[Train] Epoch 7/10 — Loss: 0.6222, Acc: 68.12%
[Train] Epoch 8/10 — Loss: 0.6016, Acc: 70.20%
[Train] Epoch 9/10 — Loss: 0.5664, Acc: 74.04%
[Train] Epoch 10/10 — Loss: 0.5658, Acc: 73.66%
Modelo final guardado en dock_scratch_noaug.pth
[Test] Loss: 0.8246, Acc: 43.24%


In [12]:
print("\n Modelo sin pesos y con data augmentation\n")
train(dock_scratch_aug,      dc_train_loader_aug, NUM_EPOCHS, model_name="dock_scratch_aug",      lr=LR)
test_model   (dock_scratch_aug,       dc_test_loader,      model_name="dock_scratch_aug")


 Modelo sin pesos y con data augmentation

[Train] Epoch 1/10 — Loss: 0.6958, Acc: 48.27%
[Train] Epoch 2/10 — Loss: 0.6933, Acc: 51.92%
[Train] Epoch 3/10 — Loss: 0.6893, Acc: 53.34%
[Train] Epoch 4/10 — Loss: 0.7022, Acc: 52.77%
[Train] Epoch 5/10 — Loss: 0.6907, Acc: 54.91%
[Train] Epoch 6/10 — Loss: 0.6916, Acc: 52.44%
[Train] Epoch 7/10 — Loss: 0.6901, Acc: 51.21%
[Train] Epoch 8/10 — Loss: 0.6738, Acc: 58.03%
[Train] Epoch 9/10 — Loss: 0.6974, Acc: 57.84%
[Train] Epoch 10/10 — Loss: 0.7127, Acc: 45.04%
Modelo final guardado en dock_scratch_aug.pth
[Test] Loss: 0.6953, Acc: 43.24%


In [13]:
print("\n Modelo con pesos y sin data augmentation\n")
train(dock_pretrained_noaug, dc_train_loader,     NUM_EPOCHS, model_name="dock_pretrained_noaug", lr=LR)
test_model   (dock_pretrained_noaug, dc_test_loader,      model_name="dock_pretrained_noaug")


 Modelo con pesos y sin data augmentation

[Train] Epoch 1/10 — Loss: 0.6874, Acc: 55.24%
[Train] Epoch 2/10 — Loss: 0.6116, Acc: 85.36%
[Train] Epoch 3/10 — Loss: 0.5448, Acc: 94.79%
[Train] Epoch 4/10 — Loss: 0.4962, Acc: 90.05%
[Train] Epoch 5/10 — Loss: 0.4235, Acc: 93.04%
[Train] Epoch 6/10 — Loss: 0.3731, Acc: 92.32%
[Train] Epoch 7/10 — Loss: 0.3243, Acc: 92.13%
[Train] Epoch 8/10 — Loss: 0.2505, Acc: 95.64%
[Train] Epoch 9/10 — Loss: 0.2008, Acc: 97.20%
[Train] Epoch 10/10 — Loss: 0.1593, Acc: 100.00%
Modelo final guardado en dock_pretrained_noaug.pth
[Test] Loss: 0.2833, Acc: 91.89%


In [14]:
print("\n Modelo con pesos y con data augmentation\n")
train(dock_pretrained_aug,   dc_train_loader_aug, NUM_EPOCHS, model_name="dock_pretrained_aug",   lr=LR)
test_model   (dock_pretrained_aug,    dc_test_loader,      model_name="dock_pretrained_aug")


 Modelo con pesos y con data augmentation

[Train] Epoch 1/10 — Loss: 0.6985, Acc: 44.76%
[Train] Epoch 2/10 — Loss: 0.6727, Acc: 67.27%
[Train] Epoch 3/10 — Loss: 0.6559, Acc: 75.27%
[Train] Epoch 4/10 — Loss: 0.6620, Acc: 66.89%
[Train] Epoch 5/10 — Loss: 0.6342, Acc: 74.04%
[Train] Epoch 6/10 — Loss: 0.6314, Acc: 72.42%
[Train] Epoch 7/10 — Loss: 0.5917, Acc: 76.84%
[Train] Epoch 8/10 — Loss: 0.5708, Acc: 80.48%
[Train] Epoch 9/10 — Loss: 0.5591, Acc: 79.96%
[Train] Epoch 10/10 — Loss: 0.5305, Acc: 79.44%
Modelo final guardado en dock_pretrained_aug.pth
[Test] Loss: 0.4517, Acc: 83.78%
